In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Judds,Santa Claus Is Comin' To Town,Oh you better watch out you better not cry \r...
1,"Harry Connick, Jr.",Jingle Bells,Dashing through the snow \r\nIn a one horse o...
2,Linkin Park,Lying From You,When I pretend \r\nEverything is what I want ...
3,Horrible Histories,Mary Seacole Song,"Me name Mary Seacole, famous nurse \r\nTo de ..."
4,Twenty One Pilots,Johnny Boy,He stays home from work this time \r\nHe neve...
5,OneRepublic,Trap Door,You say you're a minute from the deep end \r\...
6,Cliff Richard,Never Even Thought,Never even thougt it would happen to me \r\nM...
7,Roxette,Dreaming,Hot summer night. \r\nThe art of make-believe...
8,Ray Boltz,Someone Stood Up For Stephen,Someone Stood Up For Stephen \r\nWords and mu...
9,Mazzy Star,Unreflected,Once it's life's fortunate \r\nIsn't this so ...


In [9]:
df['text'][0]

"Oh you better watch out you better not cry  \r\nYou better not pout I'm tellin' you why  \r\nSanta Claus is comin' to town  \r\nHe's a makin' a list checkin' in twice  \r\nGonna find out who's naughty and nice  \r\nSanta Claus is comin' to town  \r\n  \r\nHe sees you when you're sleeping he knows when you're awake  \r\nHe knows if you've been bad or good so be good for goodness sake  \r\n  \r\nOh you better watch out you better not cry  \r\nYou better not pout I'm tellin' you why  \r\nSanta Claus is comin' to town  \r\n  \r\nWith little tin horns and a little toy drums  \r\nA root a toot toot and a rum a tum tums  \r\nSanta Claus is comin' to town  \r\nAnd curly head dolls that cuddle and coo  \r\nElephants boats and kiddie cars too  \r\nSanta Claus is comin' to town  \r\n  \r\nThe kids and girls in boyland will have a jubilee  \r\nThey're gonna build a toyland all around the Christmas tree  \r\n  \r\n[ guitar ]  \r\nSo you better watch out you better not cry  \r\nYou better not pout 

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.00000000e+00, 2.21036675e-03, 4.71169022e-04, ...,
       8.38063979e-03, 1.28295925e-02, 9.05836480e-03])

In [18]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [19]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [20]:
recommendation('Hello')

['Hello Walls',
 'Smells Like Teen Spirit (Live)',
 "This Guy's In Love With You",
 'Crazy About You',
 'Hello Like Before',
 'Hello Good Morning',
 'Smells Like Teen Spirit',
 'I Will',
 'Trees',
 "You Can't Win Them All",
 'Me Without You',
 'One, Two',
 'Never Even Thought',
 'Is This Love',
 'Head And Heart',
 "I've Got That Feeling",
 "Don't Let Go",
 'Breaking Your Own Heart',
 "Don't Let Me Be Lonely Tonight",
 'If I Love Again']

In [21]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))